In [3]:
import re
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import numpy
import csv

import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import multi_gpu_model



# 1. 영화의 고유 번호와 장르를 읽어오는 단입니다, 호러, 로맨스, 액션 등등 원하는 장르의 영화만 골라올 수 있습니다.
# 2. 중복되는 영화 장르가 있으므로 (Action, Comedy, Horror) 영화 장르간 우선권을 두어야 합니다. [genres[i].find("Romance") == -1 and ...]
# 3. 이 분류 방법은 한계점이 있습니다, 
#   같은 (Horror, Comedy) 장르의 영화가 있을 때, Horror 중심의 코미디를 가미한 영화가 있는 반면 \
#   Comedy 중심의 호러를 가미한 영화가 있는데, 이 딥러닝 모델은 해당 사실까지는 판별하지 못합니다.
#   오로지 해당 영화가 얼마나 잔인할까, 아니면 공포스러울가를 맞추는 데에 촛점이 맞추어져 있습니다. (정확히는 대표작들과 비슷한가)
def sequenceGenres():
    f = open(r'./Basic_DATA/Movie_Genres.tsv', 'r', encoding='utf-8')
    rdr = csv.reader(f, delimiter='\t')

    #영화, 출력 tconst, 출력 장르값
    movie, output_tconst, output_genres = [] ,[], []
    for i in rdr:
        movie.append(i)

    for i in range(movie.__len__()):
    #   if movie[i][1].find("Horror") == -1 and movie[i][1].find("Romance")>=0:

        if movie[i][1].find("Horror") >= 0:
            output_tconst.append(movie[i][0])
            output_genres.append(movie[i][1])
    
    print(output_tconst.__len__(), output_genres.__len__())
    
    return output_tconst, output_genres

# 트레인과 테스트 쓸모없는 값을 제거하는 단입니다, 현재 불용어를 제거하지 않은 상태입니다 (Movie, File)
# 불용어를 제거하게 되면     t = Tokenizer(num_words=1000) 값을 더 줄이더라도 유의미한 값을 얻을 수 있을 것으로 예상됩니다.
# t = Tokenizer(num_words=1000) 값을 줄이게 되면 GPU의 메모리 부하량이 줄어듭니다.
#추 후 라이브러리를 통해 불용어를 제거할 예정입니다.
def garbage_remove(tconst):
    train = pd.read_csv(r'./Horror set/Horror.csv', skiprows=1, names=['Text', 'sentiment'])
    test = pd.read_csv(r"./Web_Crowling/" + tconst + ".tsv",sep='\t', names=['Rank', 'ID', 'Title','Text','sentiment'])
    print(tconst)
    A, B = [], []

    for i in train['Text']:
        i = re.sub('[^a-zA-Z0-9 \n]', '', str(i))
        i = i.replace("br","").replace("  "," ").replace("   "," ")
        A.append(i)

    for i in test['Text']:
        i = re.sub('[^a-zA-Z0-9 \n]', '', str(i))
        i = i.replace("br","").replace("  "," ").replace("   "," ")
        B.append(i)

    train['Text'] = A
    test['Text'] = B
    return train,test


#쓰레기 값을 제거한 데이터들에 대해서, 백터화-웟 핫 인코딩을 한 뒤, 반환합니다.
def preprocessing(garbage_remove):
    t = Tokenizer(num_words=1250)
    train,test = garbage_remove
    test_label, train_label = [], []

    t.fit_on_texts(train['Text'])
    t.fit_on_texts(test['Text'])

    #cruel = 1, astonish = 0 로 설정합니다.
    for i in train['sentiment']:
        if i == 'cruel':
            train_label.append(1)
        else:
            train_label.append(0)
    
    #(이 부분 고려 필요) test_label 을 모두 cruel 로 설정, 정확도가 떨어질수록 astonish에 가깝다는 논지로, 상당히 정확합니다.
    # 다만 .h5파일로 저장하여 불러들인뒤, 다른 데이터 셋(B)을 추가하여 확인하면
    # 다른 데이터셋 (B)를 Test 데이터로 둔 것보다 훨신 떨어지는 결과를 얻습니다.
    for i in test['sentiment']:
            test_label.append(1)

    train_result = t.texts_to_sequences(train['Text'])
    test_result = t.texts_to_sequences(test['Text'])

    # 원 핫 인코딩
    train_result = t.sequences_to_matrix(train_result, mode='binary')
    test_result = t.sequences_to_matrix(test_result, mode='binary')
    return train_result, test_result, train_label, test_label

#실질적인 모델 구성입니다. CNN 모델로 수성되어 있습니다
def AI(preprocessing):
    X_train,X_test,y_train,y_test = preprocessing

    top_words = 2000
    max_words = 1250
    # amx_words 를 1000 이상으로 설정해 주어야 비교적 정확한 값을 얻을 수 있습니다. 임의적으로 줄이면 그래픽카드 메모리가 적어도 상관은 없지만.
    # 30~70 사이의 값은 잔차가 굉장히 많이 튀게 됩니다.
    # create the model
    model = Sequential()
    model.add(Embedding(top_words, 32, input_length=max_words))
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #model = multi_gpu_model(model, gpus=2) # 멀티 GPU를 사용하는 단입니다.
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    # EarlyStopping 을 설정하여 모델을 조기에 종료 시킬 수 있습니다.
    early_stopping = EarlyStopping(monitor='accuracy', min_delta=0, patience=3, verbose=0, mode='auto') # 조기종료 콜백함수 정의
    hist = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=128, verbose=1, callbacks=[early_stopping])
    
    # 말씀드린 .h5파일입니다. 해당파일을 START_AND_END.py 에서 실행했을때 데이터를 바꾸어 보면 (tt000000 값) 학습할 결과와 동일하게 나오지 않습니다.
    # 추측되는 바로는 keras의 Tokenizer 문제일 가능성이 있습니다.
    # 원래 이게 정상이고 제가 잘못 이해하고 있을지도 모릅니다.
    # model.save('Horror 훈련셋\\horror_model.h5')
    
    # Final evaluation of the model
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    
    return scores


def write(tconst, genres, scores):
    f = open(r"./Basic_DATA/Horror.tsv", 'a', encoding='UTF8')
    data = str(tconst) + "\t" + str(genres) + "\t" + "Horror" + "\t" + str(numpy.round(scores[1]*100,2)) + "\n"
    print("data = ",data)
    f.write(data)
    f.close()



#score = AI(preprocessing(garbage_remove()))
TCONST,GENRES = sequenceGenres()

for i in range(607,len(TCONST)):
    tconst = TCONST[i]
    genres = GENRES[i]
    scores = AI(preprocessing(garbage_remove(tconst)))
    keras.backend.clear_session()
    write(tconst, genres, scores)



1237 1237
tt0804552






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1250, 32)          64000     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1250, 32)          3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 625, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               5000250   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 251       
Total params: 5,067,60

/home/cyp/Develop/anaconda/lib/python3.7/site-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


 5888/11790 [=============>................] - ETA: 3s - loss: 0.1249 - acc: 0.9521

KeyboardInterrupt: 

In [6]:
import re
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import numpy
import csv

import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import multi_gpu_model



# 1. 영화의 고유 번호와 장르를 읽어오는 단입니다, 호러, 로맨스, 액션 등등 원하는 장르의 영화만 골라올 수 있습니다.
# 2. 중복되는 영화 장르가 있으므로 (Action, Comedy, Horror) 영화 장르간 우선권을 두어야 합니다. [genres[i].find("Romance") == -1 and ...]
# 3. 이 분류 방법은 한계점이 있습니다, 
#   같은 (Horror, Comedy) 장르의 영화가 있을 때, Horror 중심의 코미디를 가미한 영화가 있는 반면 \
#   Comedy 중심의 호러를 가미한 영화가 있는데, 이 딥러닝 모델은 해당 사실까지는 판별하지 못합니다.
#   오로지 해당 영화가 얼마나 잔인할까, 아니면 공포스러울가를 맞추는 데에 촛점이 맞추어져 있습니다. (정확히는 대표작들과 비슷한가)
def sequenceGenres():
    f = open(r'./Basic_DATA/Movie_Genres.tsv', 'r', encoding='utf-8')
    rdr = csv.reader(f, delimiter='\t')

    #영화, 출력 tconst, 출력 장르값
    movie, output_tconst, output_genres = [] ,[], []
    for i in rdr:
        movie.append(i)

    for i in range(movie.__len__()):
        if movie[i][1].find("Horror") == -1 and movie[i][1].find("Romance")>=0:
            output_tconst.append(movie[i][0])
            output_genres.append(movie[i][1])
    print(output_tconst.__len__(), output_genres.__len__())
    
    return output_tconst, output_genres

# 트레인과 테스트 쓸모없는 값을 제거하는 단입니다, 현재 불용어를 제거하지 않은 상태입니다 (Movie, File)
# 불용어를 제거하게 되면     t = Tokenizer(num_words=1000) 값을 더 줄이더라도 유의미한 값을 얻을 수 있을 것으로 예상됩니다.
# t = Tokenizer(num_words=1000) 값을 줄이게 되면 GPU의 메모리 부하량이 줄어듭니다.
#추 후 라이브러리를 통해 불용어를 제거할 예정입니다.
def garbage_remove(tconst):
    train = pd.read_csv(r'./Romance set/Romance.csv', skiprows=1, names=['Text', 'sentiment'])
    test = pd.read_csv(r"Web_Crowling/" + tconst + ".tsv",sep='\t', names=['Rank', 'ID', 'Title','Text','sentiment'])
    print(tconst)
    A, B = [], []

    for i in train['Text']:
        i = re.sub('[^a-zA-Z0-9 \n]', '', str(i))
        i = i.replace("br","").replace("  "," ").replace("   "," ")
        A.append(i)

    for i in test['Text']:
        i = re.sub('[^a-zA-Z0-9 \n]', '', str(i))
        i = i.replace("br","").replace("  "," ").replace("   "," ")
        B.append(i)

    train['Text'] = A
    test['Text'] = B
    return train,test


#쓰레기 값을 제거한 데이터들에 대해서, 백터화-웟 핫 인코딩을 한 뒤, 반환합니다.
def preprocessing(garbage_remove):
    t = Tokenizer(num_words=1250)
    train,test = garbage_remove
    test_label, train_label = [], []

    t.fit_on_texts(train['Text'])
    t.fit_on_texts(test['Text'])

    #cruel = 1, astonish = 0 로 설정합니다.
    for i in train['sentiment']:
        if i == 'happy':
            train_label.append(1)
        else:
            train_label.append(0)
    
    #(이 부분 고려 필요) test_label 을 모두 cruel 로 설정, 정확도가 떨어질수록 astonish에 가깝다는 논지로, 상당히 정확합니다.
    # 다만 .h5파일로 저장하여 불러들인뒤, 다른 데이터 셋(B)을 추가하여 확인하면
    # 다른 데이터셋 (B)를 Test 데이터로 둔 것보다 훨신 떨어지는 결과를 얻습니다.
    for i in test['sentiment']:
            test_label.append(1)

    train_result = t.texts_to_sequences(train['Text'])
    test_result = t.texts_to_sequences(test['Text'])

    # 원 핫 인코딩
    train_result = t.sequences_to_matrix(train_result, mode='binary')
    test_result = t.sequences_to_matrix(test_result, mode='binary')
    return train_result, test_result, train_label, test_label

#실질적인 모델 구성입니다. CNN 모델로 수성되어 있습니다
def AI(preprocessing):
    X_train,X_test,y_train,y_test = preprocessing

    top_words = 2000
    max_words = 1250
    # amx_words 를 1000 이상으로 설정해 주어야 비교적 정확한 값을 얻을 수 있습니다. 임의적으로 줄이면 그래픽카드 메모리가 적어도 상관은 없지만.
    # 30~70 사이의 값은 잔차가 굉장히 많이 튀게 됩니다.
    # create the model
    model = Sequential()
    model.add(Embedding(top_words, 32, input_length=max_words))
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #model = multi_gpu_model(model, gpus=2) # 멀티 GPU를 사용하는 단입니다.
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    # EarlyStopping 을 설정하여 모델을 조기에 종료 시킬 수 있습니다.
    early_stopping = EarlyStopping(monitor='accuracy', min_delta=0, patience=3, verbose=0, mode='auto') # 조기종료 콜백함수 정의
    hist = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=128, verbose=1, callbacks=[early_stopping])
    
    # 말씀드린 .h5파일입니다. 해당파일을 START_AND_END.py 에서 실행했을때 데이터를 바꾸어 보면 (tt000000 값) 학습할 결과와 동일하게 나오지 않습니다.
    # 추측되는 바로는 keras의 Tokenizer 문제일 가능성이 있습니다.
    # 원래 이게 정상이고 제가 잘못 이해하고 있을지도 모릅니다.
    # model.save('Horror 훈련셋\\horror_model.h5')
    
    # Final evaluation of the model
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))
    
    return scores


def write(tconst, genres, scores):
    f = open(r"./Basic_DATA/Romance.tsv", 'a', encoding='UTF8')
    data = str(tconst) + "\t" + str(genres) + "\t" + "Romance" + "\t" + str(numpy.round(scores[1]*100,2)) + "\n"
    print("data = ",data)
    f.write(data)
    f.close()



#score = AI(preprocessing(garbage_remove()))
TCONST,GENRES = sequenceGenres()

for i in range(1315,len(TCONST)):
    tconst = TCONST[i]
    genres = GENRES[i]
    scores = AI(preprocessing(garbage_remove(tconst)))
    keras.backend.clear_session()
    write(tconst, genres, scores)



1778 1778
tt1454505
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1250, 32)          64000     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1250, 32)          3104      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 625, 32)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 250)               5000250   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 251       
Total params: 5,067,605
Trainable params: 5,067,605
Non-trainable params: 0
______________________________________________

KeyboardInterrupt: 

In [1]:
import tensorflow as tf
# 그래프를 생성합니다.
c = []
for d in ['/gpu:0', '/gpu:1']:
    with tf.device(d):
        a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3])
        b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2])
        c.append(tf.matmul(a, b))
with tf.device('/cpu:0'):
    sum = tf.add_n(c)
# log_device_placement을 True로 설정하여 세션을 만듭니다.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# op를 실행합니다.
sess.run(sum)

array([[ 44.,  56.],
       [ 98., 128.]], dtype=float32)